#  create NOAA DSDP taxa list for import
## 1-96 taxa

create taxa list to import in database

In [55]:
import sys
import csv
import glob
import os
import requests
import re
import time 

sys.path.append('../../')
import pandas as pd
import numpy as np

# import db 
import scripts.normalize_taxa as nt
from config import OUTPUT_DIR, CLEAN_DATA_DIR, RAW_DATA_DIR
from scripts.pbdb import get_parent_taxa, PBDB_TAXA_ID, PBDB_TAXA_NAME
from scripts.shared_utils import (
    log_df
)
import scripts.pbdb as pbdb

In [182]:

date='2024-04-03'


base_dir = CLEAN_DATA_DIR

initial_input_file = RAW_DATA_DIR/'PI_processed_files'/'NOAA'/f'NOAA_taxa_lists_taxa_list_{date}.csv'
input_file = OUTPUT_DIR/'taxa'/'NOAA'/f"PI_normalized_taxa_list_with_pbdb_{date}.csv"

input_file_species_temp = OUTPUT_DIR/'taxa'/'NOAA'/f"PI_normalized_taxa_list_with_species_temp_{date}.csv"
input_file_species = OUTPUT_DIR/'taxa'/'NOAA'/f"PI_normalized_taxa_list_with_species_{date}.csv"

crosswalk_file = OUTPUT_DIR/'taxa'/'NOAA'/f"taxa_crosswalk_{date}.csv"
taxa_file = OUTPUT_DIR/'taxa'/'NOAA'/f"taxa_list_{date}.csv"

genus_path = OUTPUT_DIR/'taxa'/'NOAA'/f'genus_pbdb_{date}.csv'
higher_taxa_pbdb_file = OUTPUT_DIR/'taxa'/'NOAA'/f'higher_taxa_pbdb_{date}.csv'  
species_path = OUTPUT_DIR/'taxa'/'NOAA'/f'species_pbdb_{date}.csv'

taxa_pbdb_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'taxa_list_pbdb_{date}.csv'

# old_genus_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'genus_pbdb_{prev_date}.csv'
# old_higher_taxa_pbdb_file = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'higher_taxa_pbdb_{prev_date}.csv'  



## cleanup taxa list


In [121]:
PI_df = pd.read_csv(initial_input_file, dtype=str)

log_df(PI_df)
# (7763, 39)

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [122]:
PI_df.columns

Index(['taxon_group', 'verbatim_name', 'name', 'Comment',
       'Notes (change to Internal only notes?)',
       'Any taxon above genus modifier', 'Any taxon above genus',
       'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name',
       'species modifier', 'species name', 'subspecies modifier',
       'subspecies name', 'non-taxa descriptor', 'comments',
       'pbdb_taxon_id_old', 'pbdb_taxon_name', 'pbdb_taxon_rank',
       'Corrections to pbdb_taxon_rank', 'pbdb_taxon_id', 'pbdb_taxon_name.1',
       'pbdb_taxon_rank.1', 'Corrections to pbdb_taxon', 'family_taxon_id',
       'family_taxon_name', 'superfamily_taxon_id', 'superfamily_taxon_name',
       'order_taxon_id', 'order_taxon_name', 'class_taxon_id',
       'class_taxon_name', 'phylum_taxon_id', 'phylum_taxon_name',
       'kingdom_taxon_id', 'kingdom_taxon_name', 'unranked clade_taxon_id',
       'unranked clade_taxon_name'],
      dtype='object')

In [123]:
if 'pbdb_taxon_name.1' in PI_df:
    print('update cols')

    PI_df = PI_df.rename(columns={ 
        # 'pbdb_taxon_id': 'pbdb_taxon_id_old',
        'pbdb_taxon_name': 'pbdb_taxon_name_old',
        'pbdb_taxon_rank': 'pbdb_taxon_rank_old',
        # 'pbdb_taxon_id.1': 'pbdb_taxon_id',
        'pbdb_taxon_name.1': 'pbdb_taxon_name',
        'pbdb_taxon_rank.1': 'pbdb_taxon_rank'
    })

update cols


2. QA corrections to taxa id

In [124]:
cols = [
    'Corrections to pbdb_taxon_rank',
    'Corrections to pbdb_taxon',
    'pbdb_taxon_id_old',
    'pbdb_taxon_id'
]
tmp = PI_df[PI_df['Corrections to pbdb_taxon_rank'].notna() | 
            PI_df['Corrections to pbdb_taxon'].notna()].copy()

tmp[cols].drop_duplicates()

,Corrections to pbdb_taxon_rank,Corrections to pbdb_taxon,pbdb_taxon_id_old,pbdb_taxon_id
295,NaN,1059,383509,1059
299,genus; id 1064,1064,168467,1064
438,"genus, id 1124",NaN,70260,1124
1169,genus taxon_no=2092,2092,153914,2092
1411,genus taxon_no=2542,NaN,137071,2542
1640,NaN,441243,374615,441243
1752,genus; ID 71247,NaN,9512,71247
1989,genus; ID 82145,NaN,14728,82145
2144,478034,478034,NaN,478034
2195,441540,441540,83895,441540


In [125]:
PI_df.shape
# 7763

(7763, 39)

In [126]:
PI_df.to_csv(input_file, index=False)


## More QA

In [127]:
df = pd.read_csv(input_file, dtype=str)
df.shape
# (7763, 39)

(7763, 39)

In [128]:
df[df['pbdb_taxon_id'].isna()].shape

(17, 39)

In [129]:
df[df['pbdb_taxon_id'].isna() & 
   df['Any taxon above genus'].isna() &
  df['genus name'].isna()]

,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name


2. taxa with missing pbdb taxon id that needs to be added

In [130]:
df[df['Any taxon above genus'] == 'Globigerinidae indet.']


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
4431,planktic_forams,Forams planktonic indent.,Globigerinidae indet.,NaN,Joc manually filled in the taxon nos for this,NaN,Globigerinidae indet.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [131]:
# pbdb.fix_taxa_for_row_id(df, 4431, 82191)

3. update taxa with missing pbdb taxon name and rank

In [132]:
tmp = df[df['pbdb_taxon_name'].isna() & df['pbdb_taxon_id'].notna()] 
tmp.shape

(61, 39)

In [133]:
for index, row in tmp.iterrows():
    if row['pbdb_taxon_id']== '454654':
        continue
    pbdb.fix_taxa_for_row_id(df, index, row['pbdb_taxon_id'])


440987
478034
443687
443687
443806
443806
443806
443806
443806
276935
276935
276935
264667
477767
477767
409479
409481
409479
409479
409479
410976
410976
424279
424279
424279
424279
424279
424279
87836
453595
453595
270678
256803
24409
2825
2825
249963
308943
252756
454352
252828
478723
345975
338622
277351
418507
250494
483433
277167
259514
159808
159808
428
625
159912
650
85877
85877
85877
454938


3b. manually add taxon name and rank for taxa where the taxa lineage has changed 

In [134]:
tmp = df[df['pbdb_taxon_name'].isna() & df['pbdb_taxon_id'].notna()] 
tmp

,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name,corrected
6835,radiolarians,Pterocyoticlius barbadense,Pterocyrtidium barbadense,misspelled genus; Mikrotax,misspelling according to microtax https://www....,NaN,NaN,NaN,Pterocyrtidium,NaN,...,NaN,4,Radiolaria,87643,Chromista,NaN,NaN,NaN,NaN,NaN


In [135]:
df[df['pbdb_taxon_name'].isna() & (df['pbdb_taxon_id']=='454654')]
df.at[6835, 'pbdb_taxon_name']='Pterocyrtidium'
df.at[6835, 'pbdb_taxon_rank']='genus'


4. update pbdb data for taxa with outdated info

In [136]:
df[df['pbdb_taxon_id']=='443687']

,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name,corrected
2950,dinoflagellates,Ascostomocystis potane,NaN,NaN,"in PBDB, so it will be classified correctly wi...",NaN,NaN,NaN,Ascostomocystis,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,443687,Ascostomocystis,True
2951,dinoflagellates,Ascostomocystis sp.,NaN,NaN,NaN,NaN,NaN,NaN,Ascostomocystis,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,443687,Ascostomocystis,True


In [137]:
pbdb.fix_taxa_for_row_id(df, 2950, 443687, True)
pbdb.fix_taxa_for_row_id(df, 2951, 443687, True)

443687
443687


In [150]:
ranks = list(df['pbdb_taxon_rank'].unique())

In [151]:
for rank in ranks:
    if f'{rank}_taxon_id' in df:
        subset = (df['pbdb_taxon_rank'] == rank) & (df[f'{rank}_taxon_id'].isna())
        print(df[subset].shape, rank)

        df.loc[subset, f'{rank}_taxon_id'] = df['pbdb_taxon_id']
        df.loc[subset, f'{rank}_taxon_name'] = df['pbdb_taxon_name']
    

(0, 42) genus
(16, 42) family
(0, 42) phylum
(0, 42) unranked clade
(0, 42) class
(0, 42) superfamily
(3, 42) order


In [152]:
for rank in ranks:
    if f'{rank}_taxon_id' in df:
        subset = (df['pbdb_taxon_rank'] == rank) & (df[f'{rank}_taxon_id'].isna())
        print(df[subset].shape, rank)

(0, 42) genus
(0, 42) family
(0, 42) phylum
(0, 42) unranked clade
(0, 42) class
(0, 42) superfamily
(0, 42) order


In [ ]:
del df['corrected']

In [153]:

df.to_csv(input_file, index=False)


## create species file

In [154]:
PI_df = pd.read_csv(input_file, dtype=str)
log_df(PI_df)
# (7763, 39)

(7763, 42)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name,corrected
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,762,Abyssamina,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN


In [155]:
pbdb.add_genus_species(PI_df)
log_df(PI_df)
# (7763, 40)

(7763, 43)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name,corrected,genus species
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,762,Abyssamina,NaN,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,Alabamina haitiensis


In [23]:
species_df = PI_df[PI_df['genus species'].notna()].copy()[['taxon_group',  'genus species']]
species_df.drop_duplicates(inplace=True)

log_df(species_df)
# (5451, 2)

(5451, 2)


,taxon_group,genus species
0,benthic_forams,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum
3,benthic_forams,Alabamina decorata
4,benthic_forams,Alabamina haitiensis
5,benthic_forams,Alabamina marginata


In [24]:
pbdb.fetch_pdbd_data(species_df, 'genus species', search_type="name")

0 50 100 150 200 250 300 350 400 450 500 550 650 700 750 800 850 900 1000 1050 1100 1150 1200 1250 1300 1450 1500 1550 1600 1650 1700 1800 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2950 3100 3150 3200 3250 3300 3350 3500 3550 3650 3700 3750 3800 3850 3900 3950 4000 4050 4100 4200 4250 4350 4500 4550 4600 4650 4800 4850 4900 4950 5000 5050 5150 5200 5250 5450 5500 5550 5800 5850 5950 6000 6050 6100 6150 6250 6350 6500 6550 6650 6700 6750 6800 6850 6900 6950 7000 7050 7100 7150 7300 7350 7400 7700 7750 

In [ ]:
del species_df['check']
log_df(species_df)
# (5451, 19)

In [31]:
species_df.to_csv(species_path, index=False)

## add species pbdb info to input file

In [159]:
species_df = pd.read_csv(species_path, dtype=str)

log_df(species_df)
# (5451, 19)

(5451, 19)


,taxon_group,genus species,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,kingdom_taxon_id,kingdom_taxon_name,phylum_taxon_id,phylum_taxon_name,class_taxon_id,class_taxon_name,order_taxon_id,order_taxon_name,family_taxon_id,family_taxon_name,genus_taxon_id,genus_taxon_name,species_taxon_id,species_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,benthic_forams,Alabamina marginata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
filter_species_df = species_df[species_df['pbdb_taxon_name'].notna() & (species_df['pbdb_taxon_rank'].isin(['species', 'subspecies']))]
log_df(filter_species_df)
# (556, 19)

(556, 19)


,taxon_group,genus species,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,kingdom_taxon_id,kingdom_taxon_name,phylum_taxon_id,phylum_taxon_name,class_taxon_id,class_taxon_name,order_taxon_id,order_taxon_name,family_taxon_id,family_taxon_name,genus_taxon_id,genus_taxon_name,species_taxon_id,species_taxon_name
5,benthic_forams,Alabamina midwayensis,477339,Alabamina midwayensis,species,87643,Chromista,71228,Foraminifera,428504,Globothalamea,279579,Rotaliida,82213,Alabaminidae,788,Alabamina,477339,Alabamina midwayensis
18,benthic_forams,Ammodiscus incertus,385961,Ammodiscus incertus,species,87643,Chromista,71228,Foraminifera,NaN,NaN,NaN,NaN,112199,Ammodiscidae,818,Ammodiscus,385961,Ammodiscus incertus
29,benthic_forams,Ammotium cassis,240805,Ammotium cassis,species,87643,Chromista,71228,Foraminifera,NaN,NaN,428780,Lituolida,103760,Lituolidae,829,Ammotium,240805,Ammotium cassis
38,benthic_forams,Angulogerina occidentalis,355060,Angulogerina occidentalis,species,87643,Chromista,71228,Foraminifera,NaN,NaN,NaN,NaN,103766,Buliminidae,852,Angulogerina,355060,Angulogerina occidentalis
59,benthic_forams,Anomalinoides acuta,382640,Anomalinoides acuta,species,87643,Chromista,71228,Foraminifera,428504,Globothalamea,279579,Rotaliida,103798,Anomalinidae,859,Anomalinoides,382640,Anomalinoides acuta


In [161]:
PI_df = pd.read_csv(input_file, dtype=str)
pbdb.add_genus_species(PI_df)
PI_df.loc[PI_df['pbdb_taxon_rank'] == 'genus', 'genus_taxon_id' ] = PI_df['pbdb_taxon_id'] 
PI_df.loc[PI_df['pbdb_taxon_rank'] == 'genus', 'genus_taxon_name' ] = PI_df['pbdb_taxon_name'] 

log_df(PI_df)
# (7763, 42)

(7763, 42)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name,genus species
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,762,Abyssamina,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,Alabamina haitiensis


In [162]:
temp_cols = ['genus species', 'taxon_group', 
        # 'phylum_taxon_id', 'phylum_taxon_name',
        'class_taxon_id', 'class_taxon_name',
        'order_taxon_id',  'order_taxon_name',
        'family_taxon_id', 'family_taxon_name',
       # 'genus_taxon_id', 'genus_taxon_name',

       ]

temp_df = PI_df.merge(filter_species_df, on=temp_cols)
# 'genus species', 'taxon_group' - 966

# phylum_taxon_id - 359
# phylum_taxon_id, class_taxon_id - 312
# phylum_taxon_id, class_taxon_id, order_taxon_id - 286
# phylum_taxon_id, class_taxon_id, order_taxon_id, family_taxon_id - 279

# genus_taxon_id - 691
# family_taxon_id, genus_taxon_id - 611
# order_taxon_id,  family_taxon_id, genus_taxon_id - 578
# class_taxon_id, order_taxon_id,  family_taxon_id, genus_taxon_id - 483
# phylum_taxon_id, class_taxon_id, order_taxon_id,  family_taxon_id, genus_taxon_id - 185

#  family_taxon_id - 807
# order_taxon_id,  family_taxon_id - 769
# class_taxon_id, order_taxon_id,  family_taxon_id - 670
# phylum_taxon_id, class_taxon_id, order_taxon_id,  family_taxon_id - 279


temp_df.shape
# (670, 53)

(670, 53)

In [38]:
# foo = temp_df[temp_df['genus_taxon_id_x'] != temp_df['genus_taxon_id_y']][['pbdb_taxon_name_x','genus species', 'genus_taxon_name_x', 'genus_taxon_name_y']]
# foo.to_csv('foo.csv')

In [163]:
merge_df = PI_df.merge(filter_species_df, on=temp_cols, how='left', suffixes=(None, '_y'))
log_df(merge_df)
# (7763, 53)

(7763, 53)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,pbdb_taxon_name_y,pbdb_taxon_rank_y,kingdom_taxon_id_y,kingdom_taxon_name_y,phylum_taxon_id_y,phylum_taxon_name_y,genus_taxon_id_y,genus_taxon_name_y,species_taxon_id,species_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [164]:
merge_df[ (merge_df['species_taxon_name'].notna())].shape

(670, 53)

In [165]:
merge_df[(merge_df['phylum_taxon_name'] != merge_df['phylum_taxon_name_y']) & (merge_df['species_taxon_name'].notna())].shape

(181, 53)

In [166]:
merge_df[(merge_df['phylum_taxon_name'] == merge_df['phylum_taxon_name_y']) & (merge_df['species_taxon_name'].notna())].shape

(489, 53)

In [167]:
merge_df.loc[(merge_df['phylum_taxon_name'] != merge_df['phylum_taxon_name_y']) & (merge_df['species_taxon_name'].notna()), 'phylum match']= False
merge_df.loc[(merge_df['phylum_taxon_name'] == merge_df['phylum_taxon_name_y']) & (merge_df['species_taxon_name'].notna()), 'phylum match']= True


In [168]:
merge_df.columns

Index(['taxon_group', 'verbatim_name', 'name', 'Comment',
       'Notes (change to Internal only notes?)',
       'Any taxon above genus modifier', 'Any taxon above genus',
       'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name',
       'species modifier', 'species name', 'subspecies modifier',
       'subspecies name', 'non-taxa descriptor', 'comments',
       'pbdb_taxon_id_old', 'pbdb_taxon_name_old', 'pbdb_taxon_rank_old',
       'Corrections to pbdb_taxon_rank', 'pbdb_taxon_id', 'pbdb_taxon_name',
       'pbdb_taxon_rank', 'Corrections to pbdb_taxon', 'family_taxon_id',
       'family_taxon_name', 'superfamily_taxon_id', 'superfamily_taxon_name',
       'order_taxon_id', 'order_taxon_name', 'class_taxon_id',
       'class_taxon_name', 'phylum_taxon_id', 'phylum_taxon_name',
       'kingdom_taxon_id', 'kingdom_taxon_name', 'unranked clade_taxon_id',
       'unranked clade_taxon_name', 'genus_taxon_id', 'genus_taxon_name',
       'genus species', 'pbdb_taxon_id

In [183]:
merge_df.to_csv(input_file_species_temp, index=False)

## clean up columns in input species file

In [190]:
PI_df = pd.read_csv(input_file_species_temp, dtype=str)
log_df(PI_df)
# (7763, 41)

(7763, 54)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,pbdb_taxon_rank_y,kingdom_taxon_id_y,kingdom_taxon_name_y,phylum_taxon_id_y,phylum_taxon_name_y,genus_taxon_id_y,genus_taxon_name_y,species_taxon_id,species_taxon_name,phylum match
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [194]:
cols = [col for col in PI_df.columns if not col.endswith('_y')]
cols = [col for col in cols if col not in ['genus species', 'phylum match', 'pbdb_taxon_id_old','pbdb_taxon_name_old', 'pbdb_taxon_rank_old' ]]
cols

['taxon_group',
 'verbatim_name',
 'name',
 'Comment',
 'Notes (change to Internal only notes?)',
 'Any taxon above genus modifier',
 'Any taxon above genus',
 'genus modifier',
 'genus name',
 'subgenera modifier',
 'subgenera name',
 'species modifier',
 'species name',
 'subspecies modifier',
 'subspecies name',
 'non-taxa descriptor',
 'comments',
 'Corrections to pbdb_taxon_rank',
 'pbdb_taxon_id',
 'pbdb_taxon_name',
 'pbdb_taxon_rank',
 'Corrections to pbdb_taxon',
 'family_taxon_id',
 'family_taxon_name',
 'superfamily_taxon_id',
 'superfamily_taxon_name',
 'order_taxon_id',
 'order_taxon_name',
 'class_taxon_id',
 'class_taxon_name',
 'phylum_taxon_id',
 'phylum_taxon_name',
 'kingdom_taxon_id',
 'kingdom_taxon_name',
 'unranked clade_taxon_id',
 'unranked clade_taxon_name',
 'genus_taxon_id',
 'genus_taxon_name',
 'species_taxon_id',
 'species_taxon_name']

In [195]:
updated_df = PI_df[cols]
updated_df.shape
# (7763, 40)

(7763, 40)

In [196]:
updated_df.to_csv(input_file_species, index=False)

## create crosswalk file

In [197]:
PI_df = pd.read_csv(input_file_species, dtype=str)
log_df(PI_df)
# (7763, 40)

(7763, 40)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name,species_taxon_id,species_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,762,Abyssamina,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,NaN


In [198]:
df = nt.add_normalized_name_column(PI_df)
log_df(df)

(7763, 41)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name,species_taxon_id,species_taxon_name,normalized_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,762,Abyssamina,NaN,NaN,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,NaN,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,NaN,Adercotryma sp.
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,NaN,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,NaN,Alabamina haitiensis


In [199]:
crosswalk_df = nt.create_taxa_crosswalk_df(df)
log_df(crosswalk_df)
# 7763
# 7763
# 7762

fields: ['Any taxon above genus', 'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name', 'species modifier', 'species name', 'subspecies modifier', 'subspecies name', 'non-taxa descriptor', 'normalized_name', 'taxon_group', 'verbatim_name', 'name comment field', 'Comment', 'Notes (change to Internal only notes?)', 'comments']
initial df:  (7763, 17)
remove nontaxa df:  (7763, 17)
drop duplicates df:  (7762, 17)
(7762, 17)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments
0,,,Abyssamina,,,,incisa,,,,Abyssamina incisa,benthic_forams,Abyssamina incisa,NaN,,,
1,,,Adercotryma,,,,glomeratum,,,,Adercotryma glomeratum,benthic_forams,Adercotryma glomeratum,NaN,,,
2,,,Adercotryma,,,,sp.,,,,Adercotryma sp.,benthic_forams,Adercotryma sp.,NaN,,,
3,,,Alabamina,,,,decorata,,,,Alabamina decorata,benthic_forams,Alabamina decorata,NaN,,,
4,,,Alabamina,,,,haitiensis,,,,Alabamina haitiensis,benthic_forams,Alabamina haitiensis,NaN,,,


In [200]:
PI_df.to_csv(crosswalk_file, index=False)

## create taxa file

In [201]:
PI_df = pd.read_csv(input_file, dtype=str)
log_df(PI_df)
# (7763, 41)

(7763, 41)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,762,Abyssamina
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina


In [202]:
df = nt.add_normalized_name_column(PI_df)
log_df(df)

(7763, 42)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name,normalized_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,762,Abyssamina,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,Adercotryma sp.
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,Alabamina haitiensis


In [203]:
taxa_df = nt.create_taxa_list_df(df)
log_df(taxa_df)
# 7763
# 7763
# 7656

fields: ['Any taxon above genus', 'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name', 'species modifier', 'species name', 'subspecies modifier', 'subspecies name', 'non-taxa descriptor', 'normalized_name', 'taxon_group', 'pbdb_taxon_id', 'pbdb_taxon_name', 'pbdb_taxon_rank', 'family_taxon_id', 'family_taxon_name', 'order_taxon_id', 'order_taxon_name', 'class_taxon_id', 'class_taxon_name', 'phylum_taxon_id', 'phylum_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name']
initial df:  (7763, 25)
remove nontaxa df:  (7763, 25)
drop duplicates df:  (7656, 25)
(7656, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,,,Abyssamina,,,,incisa,,,,...,103796,Chilostomellidae,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria
1,,,Adercotryma,,,,glomeratum,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
2,,,Adercotryma,,,,sp.,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
3,,,Alabamina,,,,decorata,,,,...,82213,Alabaminidae,,,,,288974,Foraminifera,212476,Rhizaria
4,,,Alabamina,,,,haitiensis,,,,...,82213,Alabaminidae,,,,,288974,Foraminifera,212476,Rhizaria


In [204]:
taxa_df.to_csv(taxa_file, index=False)